# 🏏 IPL 2025 Squad Selection — Data-Driven Custom Team Builder

## ✨ Introduction

In this notebook, we develop a **dynamic and fully customizable IPL 2025 squad selection algorithm**.  
Users can specify exactly how many batsmen, spinners, pacers, and all-rounders they want, and allocate a separate budget for each category.  
The algorithm then intelligently picks the **highest impact players** within each role and budget constraint, ensuring a professional, balanced, and competitive squad.

## 💡 Key Features

- User-defined squad composition (exact number of players per role).
- Smart budget control for each department (batsmen, bowlers, all-rounders).
- Impact-based player ranking to ensure maximum performance potential.
- Automatic adjustment to guarantee a **15-player squad**, adding best available players if needed.
- Strict compliance with **overseas player limit** (max 5).

---

⚡ *Let's build the ultimate IPL 2025 dream team using data, not history!* 💥📊


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ipl dataset/Final Data Ipl.csv')

In [ ]:
df.head()

,Player Name,Team,Batting Impact,BI/inn,Inns,Runs,Impact Runs,Points,Average,Matches,Wickets,Economy,Type,Base,Sold,Total_Impact
0,Rishabh Pant,LSG,123.5,9.5,13,269,227.5,NaN,NaN,NaN,NaN,NaN,BAT,2,27.00,123.5
1,Shreyas Iyer,PBKS,590.8,34.7,17,604,630.7,NaN,NaN,NaN,NaN,NaN,BAT,2,26.75,590.8
2,Venkatesh Iyer,KKR,98.5,14.0,7,142,130.5,NaN,NaN,NaN,NaN,NaN,AR,2,23.75,98.5
3,Heinrich Klaasen,SRH,420.3,32.3,13,487,518.8,NaN,NaN,NaN,NaN,NaN,BAT,-,23.00,420.3
4,Nicholas Pooran,LSG,557.8,39.8,14,524,566.5,NaN,NaN,NaN,NaN,NaN,BAT,-,21.00,557.8


In [9]:
pacer_names = [
    "Mayank Yadav", "Mohsin Khan", "Avesh Khan", "Akash Deep",
    "Jofra Archer", "Tushar Deshpande", "Sandeep Sharma", "Akash Madhwal",
    "Kagiso Rabada", "Mohammed Siraj", "Prasidh Krishna", "Ishant Sharma",
    "Anrich Nortje", "Andre Russell", "Harshit Rana", "Vaibhav Arora",
    "Josh Hazlewood", "Bhuvneshwar Kumar", "Yash Dayal", "Rasikh Dar",
    "Arshdeep Singh", "Marco Jansen", "Lockie Ferguson", "Kuldeep Sen",
    "Matheesha Pathirana", "Sam Curran", "Khaleel Ahmed", "Anshul Kamboj",
    "Mohammed Shami", "Harshal Patel", "Pat Cummins", "Simarjeet Singh",
    "Jasprit Bumrah", "Trent Boult", "Deepak Chahar", "Hardik Pandya",
    "Mitchell Starc", "Mohit Sharma", "T. Natarajan", "Mukesh Kumar"
]

# Add Bowler_Type column
def assign_bowler_type(row):
    if row['Type'] == 'BOWL' or row['Type'] == 'AR':
        if row['Player Name'] in pacer_names:
            return 'Pacer'
        else:
            return 'Spinner'
    else:
        return None

df['Bowler_Type'] = df.apply(assign_bowler_type, axis=1)

In [10]:
overseas_players = [
    "Jos Buttler", "Kagiso Rabada", "Sherfane Rutherford", "Rashid Khan", "Karim Janat", "Dasun Shanaka", "Gerald Coetzee", "Kusal Mendis",
    "Jacob Bethell", "Lungi Ngidi", "Romario Shepherd", "Phil Salt", "Tim David", "Liam Livingstone", "Nuwan Thushara", "Blessing Muzarabani",
    "Will Jacks", "Ryan Rickelton", "Corbin Bosch", "Bevon Jacobs", "Mitchell Santner", "Trent Boult", "Reece Topley", "Mujeeb Ur Rahman",
    "Jonny Bairstow", "Richard Gleeson", "Marco Jansen", "Mitchell Owen", "Azmatullah Omarzai", "Xavier Bartlett", "Kyle Jamieson",
    "Tristan Stubbs", "Faf du Plessis", "Dushmantha Chameera", "Sediqullah Atal", "Mustafizur Rahman",
    "Sunil Narine", "Andre Russell", "Quinton de Kock", "Spencer Johnson", "Rahmanullah Gurbaz", "Anrich Nortje",
    "Aiden Markram", "Mitchell Marsh", "Nicholas Pooran", "David Miller", "Matthew Breetzke", "Will O’Rourke",
    "Pat Cummins", "Travis Head", "Heinrich Klaasen", "Kamindu Mendis", "Eshan Malinga",
    "Shimron Hetmyer", "Lhuan-dre Pretorius", "Fazalhaq Farooqi", "Kwena Maphaka", "Wanindu Hasaranga", "Maheesh Theekshana",
    "Noor Ahmad", "Matheesha Pathirana", "Dewald Brevis", "Devon Conway"
]

# Add Overseas column
df['Overseas'] = df['Player Name'].apply(lambda x: 'Yes' if x in overseas_players else 'No')

In [18]:
# --- Inputs from user ---
n_batsmen = int(input("Enter number of batsmen: "))
n_spinners = int(input("Enter number of spinners: "))
n_pacers = int(input("Enter number of pacers: "))
n_allrounders = int(input("Enter number of all-rounders: "))

budget_bat = float(input("Enter budget for batsmen (Cr): "))
budget_bowl = float(input("Enter budget for bowlers (Cr): "))
budget_ar = float(input("Enter budget for all-rounders (Cr): "))

max_overseas = 5
total_budget = budget_bat + budget_bowl + budget_ar

# --- Pools ---
bat_df = df[df['Type'] == 'BAT'].sort_values('Total_Impact', ascending=False).reset_index(drop=True)
ar_df = df[df['Type'] == 'AR'].sort_values('Total_Impact', ascending=False).reset_index(drop=True)
bowl_df = df[df['Type'] == 'BOWL'].sort_values('Total_Impact', ascending=False).reset_index(drop=True)

spinner_df = bowl_df[bowl_df['Bowler_Type'] == 'Spinner'].sort_values('Total_Impact', ascending=False)
pacer_df = bowl_df[bowl_df['Bowler_Type'] == 'Pacer'].sort_values('Total_Impact', ascending=False)

selected_players = []
overseas_count = 0

def select_exact_top(df_role, role_count, role_budget, overseas_count, max_overseas):
    role_players = []
    used_budget = 0.0
    for _, row in df_role.iterrows():
        price = row['Sold']
        overseas = row['Overseas']

        if overseas == 'Yes' and overseas_count >= max_overseas:
            continue
        if used_budget + price > role_budget:
            continue

        role_players.append(row.to_dict())
        used_budget += price
        if overseas == 'Yes':
            overseas_count += 1

        if len(role_players) == role_count:
            break
    return role_players, used_budget, overseas_count

# --- Select batsmen ---
bat_players, spent_bat, overseas_count = select_exact_top(bat_df, n_batsmen, budget_bat, overseas_count, max_overseas)

# --- Select spinners ---
spin_players, spent_spin, overseas_count = select_exact_top(spinner_df, n_spinners, budget_bowl/2, overseas_count, max_overseas)

# --- Select pacers ---
pace_players, spent_pace, overseas_count = select_exact_top(pacer_df, n_pacers, budget_bowl/2, overseas_count, max_overseas)

# --- Select all-rounders ---
ar_players, spent_ar, overseas_count = select_exact_top(ar_df, n_allrounders, budget_ar, overseas_count, max_overseas)

# Combine all
selected_players = bat_players + spin_players + pace_players + ar_players

# --- If less than 15 players, add best available extra players ---
if len(selected_players) < 15:
    picked_names = [p['Player Name'] for p in selected_players]
    remaining_df = df[~df['Player Name'].isin(picked_names)].sort_values('Total_Impact', ascending=False)
    for _, row in remaining_df.iterrows():
        price = row['Sold']
        overseas = row['Overseas']

        if overseas == 'Yes' and overseas_count >= max_overseas:
            continue
        if (sum(p['Sold'] for p in selected_players) + price) > total_budget:
            continue

        selected_players.append(row.to_dict())
        if overseas == 'Yes':
            overseas_count += 1

        if len(selected_players) == 15:
            break

# Final DataFrame
final_team_df = pd.DataFrame(selected_players)
final_team_df.index = range(1, len(final_team_df) + 1)

# Display
display(final_team_df[['Player Name', 'Type', 'Total_Impact', 'Sold', 'Overseas']])

# Summary
print(f"💰 Total Team Price: ₹{round(final_team_df['Sold'].sum(), 2)} Cr (Total Budget: ₹{total_budget} Cr)")
print(f"🌍 Total Overseas Players: {overseas_count}")
print(f"👥 Total Squad Size: {len(final_team_df)}")


Enter number of batsmen: 5
Enter number of spinners: 2
Enter number of pacers: 4
Enter number of all-rounders: 4
Enter budget for batsmen (Cr): 40
Enter budget for bowlers (Cr): 35
Enter budget for all-rounders (Cr): 25


,Player Name,Type,Total_Impact,Sold,Overseas
1,Suryakumar Yadav,BAT,653.3,16.35,No
2,Shubman Gill,BAT,602.9,16.50,No
3,Prabhsimran Singh,BAT,494.1,4.00,No
4,Aiden Markram,BAT,402.8,2.00,Yes
5,Ryan Rickelton,BAT,359.8,1.00,Yes
6,Sunil Narine,BOWL,626.5,12.00,Yes
7,Wanindu Hasaranga,BOWL,317.9,5.25,Yes
8,Josh Hazlewood,BOWL,563.3,12.50,No
9,Vaibhav Arora,BOWL,386.8,1.80,No
10,Lockie Ferguson,BOWL,122.5,2.00,No


💰 Total Team Price: ₹96.05 Cr (Total Budget: ₹100.0 Cr)
🌍 Total Overseas Players: 5
👥 Total Squad Size: 15


## 🏁 Conclusion

In this project, I focused entirely on data — analyzing each player's impact score and auction price to build the most optimized IPL 2025 squad possible.  
The final 15-player team was created strictly based on **impact and budget**, not on historical reputation or big names.  
I am currently working purely on data, and this team was selected only by considering numbers and performance metrics, rather than past glory or popularity.

---

**I am working on data right now, and I have built this team purely based on impact and budget, strictly according to data — not history.**

---

This approach truly reflects the mindset of a data analyst: trusting numbers, maximizing value, and making strategic decisions without bias.

---

⚡ *Thank you for reading! Let’s gear up for an impactful IPL 2025 season!* 💥🏏
